#### Drift Benchmark SQLite Preparation
This dev notebook is all about preparing driftDb.db for actual

In [ ]:
# sqlite
import sqlite3
DRIFT_DB_PATH = "..\\data\\driftDb.db"

sql_conxn = sqlite3.connect(DRIFT_DB_PATH)
sql_cursor = sql_conxn.cursor()

In [ ]:
# list all tables
def list_all_tables():
    sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
    sql_cursor.execute(sql_query)
    print("Tables: \n".format(sql_cursor.fetchall()))
    
list_all_tables()

In [ ]:
TABLE_PARAMETER = "{TABLE_PARAMETER}"
DROP_TABLE_SQL = f"DROP TABLE {TABLE_PARAMETER};"
GET_TABLES_SQL = "SELECT name FROM sqlite_schema WHERE type='table';"


def delete_all_tables(sql_conxn):
    tables = get_tables(sql_conxn)
    delete_tables(sql_conxn, tables)


def get_tables(sql_conxn):
    sql_cursor = sql_conxn.cursor()
    sql_cursor.execute(GET_TABLES_SQL)
    tables = sql_cursor.fetchall()
    return tables


def delete_tables(sql_conxn, tables):
    sql_cursor = sql_conxn.cursor()
    for table, in tables:
        sql = DROP_TABLE_SQL.replace(TABLE_PARAMETER, table)
        sql_cursor.execute(sql)
        
    
delete_all_tables(sql_conxn)    

list_all_tables()

sql_conxn.commit()

SQLite Basics

In [ ]:
# CREATE

sql_cursor.execute("CREATE TABLE movie(title, year, score)")
res = sql_cursor.execute("SELECT name FROM sqlite_master")
res.fetchone()

In [ ]:
sql_cursor.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")


In [ ]:
sql_conxn.commit()

In [ ]:

def insert_sample(chat_id, sample_dt, cos_sim, completion):
    sql = "INSERT INTO tSample (chat_id, sample_dt, cos_sim, completion ) VALUES ('{}', '{}', '{}', '{}');".format(chat_id, sample_dt, cos_sim, completion)
    print(sql)
    
insert_sample("chat_id", "sample_dt", "cos_sim", "completion")


In [ ]:
res = sql_cursor.execute("SELECT score FROM movie")
res.fetchall()

In [ ]:
data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
sql_cursor.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
sql_conxn.commit()  # Remember to commit the transaction after executing INSERT.


In [ ]:
for row in sql_cursor.execute("SELECT year, title FROM movie ORDER BY year"):
    print(row)


In [ ]:
# Which database file are you using?
import sqlite3
sql_conxn = sqlite3.connect(DRIFT_DB_PATH)
sql_cursor = sql_conxn.cursor()
sql_cursor.execute("PRAGMA database_list")
rows = sql_cursor.fetchall()

for row in rows:
    print(row[0], row[1], row[2])

sql_conxn.close()


In [ ]:
new_con = sqlite3.connect(DRIFT_DB_PATH)
new_cursor = new_con.cursor()
res = new_cursor.execute("DROP TABLE IF EXISTS movie")
res.fetchall()
res = new_cursor.execute("SELECT name FROM sqlite_master")
res.fetchone()
new_con.close()

#### create the table and functions to insert data

In [ ]:
import sqlite3
import datetime

# SQLite benchmark sample and completion functions
# user
# insert_benchmark("chat_id00000000", "bmk_prompt", "bmk_completion")
# insert_api_completion("chat_id00000000", "bmk_prompt", "bmk_completion", 0.987654321, "api_completion")

run_time = ct = datetime.datetime.now().replace(second=0, microsecond=0)

# inserts default value of 1.0 for cos_sim
def insert_benchmark(chat_id, bmk_prompt, bmk_completion):
    sql = "INSERT INTO tSample (chat_id, run_time, bmk_prompt, bmk_completion, cos_sim) VALUES ('{}', '{}', \"{}\", \"{}\", {});".format(chat_id, run_time, bmk_prompt, bmk_completion, 1.0);
    print(sql)
    sql_cursor.execute(sql)
    sql_conxn.commit()
    

def insert_api_completion(chat_id, bmk_prompt, bmk_completion, cos_sim, api_completion):
    sql = "INSERT INTO tSample (chat_id, run_time, bmk_prompt, bmk_completion, cos_sim, api_completion) VALUES ('{}', '{}', \"{}\", \"{}\", {}, \"{}\");".format(chat_id, run_time, bmk_prompt, bmk_completion, cos_sim, api_completion);
    print(sql)
    sql_cursor.execute(sql)
    sql_conxn.commit()    


try:
    sql_conxn = sqlite3.connect(DRIFT_DB_PATH)
    sql_cursor = sql_conxn.cursor()
    #sql_cursor.execute(""" PRAGMA foreign_keys = 0;""")
    # sql_cursor.execute(""" DELETE FROM tSample;""")
    # sql_cursor.execute(""" DROP TABLE tSample;""")
    sql_cursor.execute(""" CREATE TABLE tSample (
          chat_id NOT NULL
        , run_time REAL (64) NOT NULL DEFAULT ((datetime('now', 'localtime')))
        , bmk_prompt TEXT NOT NULL
        , bmk_completion TEXT NOT NULL
        , cos_sim NUMERIC NOT NULL
        , api_completion TEXT );""")

    # sql_cursor.execute(""" PRAGMA foreign_keys = 1; """)
    res = sql_cursor.execute("SELECT name FROM sqlite_master")
    res.fetchone()


    insert_benchmark("chat_id00000000", "bmk_prompt", "bmk_completion")
    insert_api_completion("chat_id00000000", "bmk_prompt", "bmk_completion", 0.987654321, "api_completion")

    res = sql_cursor.execute("SELECT * FROM tSample;")
    rows = res.fetchall()
    print(json.dumps(rows, indent=4))

    sql_cursor.execute("DELETE FROM tSample;")
    sql_conxn.commit()

    print("sqlite3 script completed.")
    
except sqlite3.Error as error:
    print("sqlite3.Error: ", error)
finally:
    if sql_conxn:
        sql_cursor.close()
        sql_conxn.close()   



In [ ]:
# have a look a the tSample table
import json
sql_conxn = sqlite3.connect(DRIFT_DB_PATH)
sql_cursor = sql_conxn.cursor()
res = sql_cursor.execute("SELECT * FROM tSample;")
rows = res.fetchall()
print(json.dumps(rows, indent=4))
sql_conxn.close()


In [ ]:
# delete all rows
sql_conxn = sqlite3.connect(DRIFT_DB_PATH)
sql_cursor = sql_conxn.cursor()
sql_cursor.execute("DELETE FROM tSample;")
sql_conxn.commit()
sql_cursor.close()
sql_conxn.close()